### For SCDA simulator

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
import hcipy
import scipy
from scipy.interpolate import griddata
import glob
import numpy as np
from PIL import Image
from astropy.io import fits
import astropy.units as u
import pandas as pd
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.gridspec as gridspec
from pastis.temporal_analysis.close_loop_analysis import req_closedloop_calc_batch
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import pastis.util as util  
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.scda_telescopes import HexRingAPLC

In [ ]:
# Define input parameters
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
sampling = 4
num_rings = 1

In [ ]:
data_dir = CONFIG_PASTIS.get('local', 'local_data_path')
repo_dir = os.path.join(util.find_repo_location(package='pastis'))

overall_dir = util.create_data_path(data_dir, telescope='7hexaplc_design')
resDir = os.path.join(overall_dir, 'matrix_numerical')
print(resDir)

# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)

In [ ]:
tel = HexRingAPLC(optics_dir, num_rings, sampling)

In [ ]:
filepath = CONFIG_PASTIS.get('LUVOIR', 'harris_data_path')
pad_orientation = np.pi/2*np.ones(tel.nseg)

In [ ]:
tel.create_segmented_harris_mirror(filepath,pad_orientation, thermal=True,mechanical=False,other=False) 
tel.harris_sm 

In [ ]:
num_actuators = tel.harris_sm.num_actuators
num_modes = 5 #n

In [ ]:
tel.harris_sm.flatten()
unaberrated_coro_psf, ref = tel.calc_psf(ref=True, display_intermediate=False, norm_one_photon=True)
norm = np.max(ref)

In [ ]:
dh_intensity = (unaberrated_coro_psf / norm) * tel.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(tel.dh_mask != 0)])
print(f'contrast floor: {contrast_floor}')

In [ ]:
# ideal_coro_psf = np.reshape(unaberrated_coro_psf, [115,115])
# ideal_psf = np.reshape(ref, [115,115])
# ind_max = np.unravel_index(np.argmax(np.abs(ideal_psf), axis=None), np.abs(ideal_psf).shape)

#fits.writeto((os.path.join(resDir, 'ideal_psf.fits')), np.abs(ideal_psf))

plt.figure(figsize=(22,9))
plt.subplot(1,2,1)
plt.title("Coronagraphic PSF", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_coro_psf)), cmap='magma')
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.colorbar()

plt.subplot(1,2,2)
plt.title("PSF without Coronagraph", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_psf)), cmap ='magma')
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='out')
plt.colorbar()
plt.tight_layout()
plt.savefig(os.path.join(resDir, 'ideal_coro_psf.png'))

In [ ]:
nonaberrated_coro_psf, ref, efield = tel.calc_psf(ref=True, display_intermediate=True, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9

print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for pp in range(0, num_actuators):
    print(f'Working on mode {pp}/{num_actuators}')
    
    # Apply calibration aberration to used mode
    harris_actuators = np.zeros(num_actuators)
    harris_actuators[pp] = (nm_aber)/2 
    tel.harris_sm.actuators  = harris_actuators
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = tel.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
pastis_matrix = np.zeros([num_actuators, num_actuators])   # create empty matrix

for i in range(0, num_actuators):
    for j in range(0, num_actuators):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref ) * np.conj(focus_fieldS[j].electric_field -Efield_ref ))
        dh_test = (test / norm) * tel.dh_mask
        contrast = np.mean(dh_test[np.where(tel.dh_mask != 0)])
        pastis_matrix[i, j] = contrast

In [ ]:
clist = [(0.1, 0.6, 1.0), (0.05, 0.05, 0.05), (0.8, 0.5, 0.1)]
blue_orange_divergent = LinearSegmentedColormap.from_list("custom_blue_orange", clist)

plt.figure(figsize=(10,8))
#norm_mat = TwoSlopeNorm(vcenter=0,vmin=-0.005*1e-7,  vmax = 0.005*1e-7)
plt.imshow(pastis_matrix, cmap=blue_orange_divergent)
plt.title(r"PASTIS matrix $M$", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label(r"in units of ${contrast\ per\ {nm}^2}$",fontsize =15)
plt.tight_layout()

plt.savefig(os.path.join(resDir, 'pastis_matrix.png'))

In [ ]:
filename_matrix = 'PASTISmatrix_n_harris_' + str(num_actuators)
hcipy.write_fits(pastis_matrix, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_Re_matrix_n_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))


filename_matrix = 'EFIELD_Im_matrix_n_harris_' + str(num_actuators)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

end_time = time.time()
print('Runtime for harris modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

In [ ]:
evals, evecs = np.linalg.eig(pastis_matrix)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(sorted_evals)
plt.semilogy()
plt.xlabel('Mode Index',fontsize=20)
plt.ylabel('Eigenvalues (in units of contrast/'r'wave$^2$)',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,
                labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',direction='in')
plt.tick_params(axis='both',which='major',length=10, width=2)

plt.savefig(os.path.join(resDir, 'eigenvalues.png'))

In [ ]:
emodes = []
eunit = 1e-9
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    harris_coeffs = eunit*sorted_evecs[:, mode]/2
    tel.harris_sm.actuators = harris_coeffs
    wf_harris_sm = tel.harris_sm(tel.wf_aper)
    emodes.append(wf_harris_sm.phase)

In [ ]:
hcipy.imshow_field(emodes[2])

In [ ]:
harris_actuators = np.zeros(num_actuators)
harris_actuators[1] = (nm_aber)/2 
tel.harris_sm.actuators  = harris_actuators
    
# Calculate coronagraphic E-field and add to lists
aberrated_coro_psf, inter = tel.calc_psf(display_intermediate=True, return_intermediate='efield',norm_one_photon=True)